# 05wk-019: MinMaxScaler를 이용한 전처리

최규빈  
2023-10-05

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-ywZaSM1Sud-5KrwZr3CwCb&si=EknsMgx3Yq6jmOoF>

# 2. Imports

In [2]:
import numpy as np
import pandas as pd
import sklearn.preprocessing 

# 3. `MinMaxScaler()`

## A. 모티브

`-` 예제자료 로드

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/MP2023/main/posts/employment.csv').loc[:7,['toeic','gpa']]
df

`-` 모형을 돌려보고 해석한 결과

``` python
u = X.toeic*0.00571598 + X.gpa*2.46520018 -8.45433334
v = 1/(1+np.exp(-u))
v # 확률같은것임
```

-   토익이 중요해? 아니면 학점이 중요해?
-   얼만큼 중요해?

`-` 모티브: 토익점수를 0-1사이로 맞추고 gpa도 0-1사이로 맞추면 해석이
쉽지 않을까?

## B. 사용방법

`-` 스케일러 생성

In [11]:
sclr = sklearn.preprocessing.MinMaxScaler()

`-` fit, transform

In [27]:
sclr.fit(df)

In [28]:
sclr.transform(df)

`-` fit_transform

In [26]:
sclr.fit_transform(df)

## C. 잘못된 사용

`-` `sclr.fit()`와 `sclr.fit_transform()`은 입력으로 2차원 자료구조를
기대한다. (그중에서도 은근히 numpy array를 기대함)

In [35]:
sclr.fit_transform(df['toeic']) # df['toeic']는 1차원 자료구조

In [39]:
sclr.fit_transform(df[['toeic']]) # df[['toeic']]는 2차원 자료구조

# 4. `minmax_scale()`

## A. 사용방법

`-` scaler를 오브젝트로 따로 만들지 않고 함수형으로 구현

In [40]:
sklearn.preprocessing.minmax_scale(df)

`-` 이것은 심지어 1차원 자료구조에도 적용가능하다.

In [43]:
sklearn.preprocessing.minmax_scale(df['toeic'])

`-` 열별로 스케일링을 하는게 아니라 행별로 스케일링을 하는 것도
가능하다. (여기서는 필요없지만..)

In [47]:
sklearn.preprocessing.minmax_scale(df,axis=1)

## B. discussions

`-` 언뜻 보기에는 `MinMaxScaler` 보다 `minmax_scale`이 좋아보이는데,
생각보다 일반적으로 `minmax_scale`을 사용하지는 않음. 이유는 아래와
같음.

1.  파이썬을 쓰는 사람들이 함수형 접근방식보다 객체지향 접근방식을
    선호한다. (이건 제생각)
2.  학습데이터와 테스트데이터의 스케일링시 동일한 변환을 유지하는
    상황에서는 `MinMaxScaler` 가 유리함.
3.  `inverse_transform` 메서드를 같은 부가기능을 제공함.

# 5. (tr,test)의 스케일링

`-` 주어진 자료가 아래와 같이 train/test로 나누어져 있다고 가정해보자.

In [108]:
X = np.array([1.0, 2.0, 3.0, 4.0, 5.0]).reshape(-1,1)
XX = np.array([1.5, 2.5, 3.5]).reshape(-1,1)

In [94]:
X,XX

## A. 잘못된 스케일링 방법 – 비효율의 문제

In [95]:
sklearn.preprocessing.minmax_scale(X)

In [96]:
sklearn.preprocessing.minmax_scale(XX)

`-` 이 방법은 전략적으로 비효율적인 문제이지, 치팅과 관련된 치명적인
잘못은 아니다.

-   만약에 어떠한 경우에 이러한 전처리 방식이 오히려 전략적이라고 판단될
    경우 사용할수도 있음.

## B. 올바른 스케일링 방법

`-` 방법1

In [101]:
sclr = sklearn.preprocessing.MinMaxScaler()
sclr.fit(X) # 

In [102]:
sclr.transform(X)

In [103]:
sclr.transform(XX)

`-` 방법2

In [105]:
sclr = sklearn.preprocessing.MinMaxScaler()
sclr.fit_transform(X)

In [107]:
sclr.transform(XX)

## C. scaled_value $\in$ \[0,1\]?

`-` 주어진 자료가 아래와 같다고 하자.

In [109]:
X = np.array([1.0, 2.0, 3.0, 4.0, 3.5]).reshape(-1,1)
XX = np.array([1.5, 2.5, 5.0]).reshape(-1,1)

In [113]:
sclr = sklearn.preprocessing.MinMaxScaler()
sclr.fit_transform(X)
sclr.transform(XX)

-   스케일링된 값이 1보다 클 수도 있다.

## D. 아주 잘못된 스케일링 방법 – 정보누수

`-` 주어진 자료가 아래와 같다고 하자.

In [121]:
X = np.array([1.0, 2.0, 3.0, 4.0, 3.5]).reshape(-1,1)
XX = np.array([1.5, 2.5, 5.0]).reshape(-1,1)

`-` 데이터를 합친다.. (미쳤어??)

In [122]:
np.concatenate([X,XX])

`-` 합친데이터에서 스케일링

In [120]:
sklearn.preprocessing.minmax_scale(np.concatenate([X,XX]))[:5]

`-` 이러한 전처리 방식을 `정보누수`라고 한다. (대회 규정에 따라서
탈락사유에 해당함)

# 6. `inverse_transform`

In [51]:
arr = sclr.fit_transform(df)
arr 

In [62]:
sclr.inverse_transform(arr).round(5)